In [2]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import  models
from visdom import Visdom

In [3]:
# 自定义数据集
class MyDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.questionnaire_data = dataframe
        self.transform = transform
    def __len__(self):
        return len(self.questionnaire_data)

    def __getitem__(self, idx):
        questionnaire = self.questionnaire_data.iloc[idx, 2:].tolist()
        label = self.questionnaire_data.iloc[idx, 1]
        sample = {'questionnaire': questionnaire, 'label': label}
        if self.transform:
            sample = self.transform(sample)
        return sample


class ToTensor(object):
    def __call__(self, sample):
        return {'questionnaire': torch.Tensor(sample['questionnaire']), 'label':sample['label']}

In [4]:
data_df = pd.read_csv("quest_only.csv", index_col=0)
train_df = data_df.sample(frac=0.8,random_state=0,axis=0)
test_df = data_df[~data_df.index.isin(train_df.index)]
train_dataset = MyDataset(train_df,transform=ToTensor())
test_dataset = MyDataset(test_df,transform=ToTensor())
train_dataloader =DataLoader(train_dataset, batch_size=10,shuffle=True, num_workers=0)
test_dataloader =DataLoader(test_dataset, batch_size=10,shuffle=True, num_workers=0)


In [5]:
class JointNet(nn.Module):
    def __init__(self,feature_extract=True, num_classes=3, hidden1=2048, hidden2=512, dropout=0.3):
        super(JointNet, self).__init__()
        model = models.vgg16(pretrained=True)
        self.features = model.features
        set_parameter_requires_grad(self.features, feature_extract)#固定特征提取层参数
        self.avgpool=model.avgpool
        self.hidden = nn.Sequential(
            nn.Linear(512*7*7 , hidden1),
            nn.ReLU(),
            nn.Linear(hidden1 , hidden2),
            nn.ReLU(),
        )
        self.classifier = nn.Sequential(
            nn.Linear(hidden2+37, hidden2+37),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(hidden2+37, num_classes)
        )
        self.questonly = nn.Sequential(
            nn.Linear(37, 64),
            nn.Dropout(p=dropout),
            nn.ReLU(),            
            nn.Linear(64, 64),
            nn.Dropout(p=dropout),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )


    def forward(self, x):
        out=self.questonly(x)
        return out
    
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=JointNet(feature_extract=False).to(device)
learning_rate=0.001
num_epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
viz = Visdom()
viz.line([0.], [0], win='quest_train_loss', opts=dict(title='train_loss'))
viz.line([[0.,0.]], [0], win='quest_acc', opts=dict(title='new_acc', legend=['train', 'test']))

Setting up a new session...


'quest_acc'

In [8]:
global_step = 0
for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    for sample_batched in train_dataloader:
        data = sample_batched['questionnaire'].to(device)
        labels = sample_batched['label'].to(device)
    

        outputs = model(data)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        

    model.eval()
    with torch.no_grad():
        for sample_batched in test_dataloader:
            data = sample_batched['questionnaire'].to(device)
            labels = sample_batched['label'].to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()

            
    print("epoch%d: loss%.5f, train_acc%.5f, test_acc%.5f" % (epoch, loss.item(),train_correct/train_total, test_correct/test_total))
    viz.line([loss.item()], [epoch], win='quest_train_loss', update='append')
    viz.line([[train_correct/train_total,test_correct/test_total]], [epoch], win='quest_acc', update='append')

epoch0: loss0.96565, train_acc0.52239, test_acc0.35294
epoch1: loss0.97873, train_acc0.56716, test_acc0.35294
epoch2: loss0.81097, train_acc0.53731, test_acc0.35294
epoch3: loss0.59787, train_acc0.55224, test_acc0.35294
epoch4: loss1.31417, train_acc0.55224, test_acc0.35294
epoch5: loss0.65297, train_acc0.58209, test_acc0.35294
epoch6: loss0.88805, train_acc0.56716, test_acc0.35294
epoch7: loss1.15435, train_acc0.62687, test_acc0.41176
epoch8: loss1.12553, train_acc0.65672, test_acc0.52941
epoch9: loss0.68451, train_acc0.73134, test_acc0.52941
epoch10: loss1.05699, train_acc0.71642, test_acc0.52941
epoch11: loss0.56246, train_acc0.71642, test_acc0.52941
epoch12: loss0.68973, train_acc0.74627, test_acc0.52941
epoch13: loss0.53882, train_acc0.73134, test_acc0.52941
epoch14: loss1.12973, train_acc0.79104, test_acc0.52941
epoch15: loss0.48136, train_acc0.77612, test_acc0.52941
epoch16: loss0.51289, train_acc0.79104, test_acc0.52941
epoch17: loss0.59152, train_acc0.79104, test_acc0.52941
ep

In [9]:
torch.save(model.state_dict(), "questmodel.pth")